# Medical Insurance Fraud Detection - Baseline Model

This notebook implements **ensemble learning models** with **traditional tabular features** for detecting fraudulent medical insurance claims.

## Features:
- **Traditional tabular representation** (basic features without advanced engineering)
- **Stacking Ensemble** combining multiple base learners

## Models Implemented:
1. **Random Forest Classifier**
2. **Gradient Boosting Classifier**
3. **XGBoost Classifier**
4. **LightGBM Classifier**
5. **Stacking Classifier** (Meta-learner ensemble)

## Workflow:
- Load medical insurance claims data
- Basic feature preprocessing
- Train multiple ensemble models with stacking
- Comprehensive evaluation and comparison

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn preprocessing and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score, f1_score, accuracy_score
)

# Ensemble models
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier, 
    StackingClassifier
)
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")

In [ ]:
# Prepare final feature matrix X and target y
feature_columns = [
    'claim_amount', 'patient_age', 'num_procedures', 'hospital_stay_days',
    'num_previous_claims', 'provider_claim_count', 'diagnosis_complexity',
    'treatment_cost_ratio', 'claim_processing_time', 'geographic_risk_score',
    'cost_per_procedure', 'cost_per_day', 'procedures_per_day'
]

# Select features and handle missing values
X = all_claims[feature_columns].copy()
X = X.fillna(X.median())

# Prepare target variable
if 'PotentialFraud' in all_claims.columns:
    y = (all_claims['PotentialFraud'] == 'Yes').astype(int)
else:
    raise ValueError("PotentialFraud column not found in the data!")

print(f"\n{'='*60}")
print(f"FINAL DATASET PREPARED")
print(f"{'='*60}")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Number of features: {len(feature_columns)}")
print(f"Fraud cases: {y.sum()} ({y.mean()*100:.2f}%)")
print(f"Legitimate cases: {(1-y).sum()} ({(1-y).mean()*100:.2f}%)")
print(f"{'='*60}")

In [ ]:
# Create baseline ratio features (3 simple features as per project design)
all_claims['cost_per_procedure'] = all_claims['claim_amount'] / (all_claims['num_procedures'] + 1)
all_claims['cost_per_day'] = all_claims['claim_amount'] / (all_claims['hospital_stay_days'] + 1)
all_claims['procedures_per_day'] = all_claims['num_procedures'] / (all_claims['hospital_stay_days'] + 1)

print("Baseline ratio features created:")
print("  ✓ cost_per_procedure")
print("  ✓ cost_per_day")
print("  ✓ procedures_per_day")

In [ ]:
# Merge with patient data to get demographics
all_claims = all_claims.merge(patients_df, on='BeneID', how='left')

# Create baseline features
print("Engineering baseline features...")

# Basic claim amount features
if 'InscClaimAmtReimbursed' in all_claims.columns:
    all_claims['claim_amount'] = all_claims['InscClaimAmtReimbursed']
elif 'ClaimAmount' in all_claims.columns:
    all_claims['claim_amount'] = all_claims['ClaimAmount']
else:
    # Calculate from deductible amounts if needed
    amount_cols = [c for c in all_claims.columns if 'Amt' in c or 'Reimbursed' in c]
    all_claims['claim_amount'] = all_claims[amount_cols].sum(axis=1) if amount_cols else 0

# Patient age (calculate from DOB if available)
if 'DOB' in all_claims.columns and 'ClaimStartDt' in all_claims.columns:
    all_claims['DOB'] = pd.to_datetime(all_claims['DOB'], errors='coerce')
    all_claims['ClaimStartDt'] = pd.to_datetime(all_claims['ClaimStartDt'], errors='coerce')
    all_claims['patient_age'] = (all_claims['ClaimStartDt'] - all_claims['DOB']).dt.days / 365.25
elif 'Age' in all_claims.columns:
    all_claims['patient_age'] = all_claims['Age']
else:
    all_claims['patient_age'] = 50  # Default if not available

# Number of procedures (count diagnosis and procedure codes)
diag_cols = [c for c in all_claims.columns if 'ClmDiagnosisCode' in c or 'DiagnosisCode' in c]
proc_cols = [c for c in all_claims.columns if 'ClmProcedureCode' in c or 'ProcedureCode' in c]
all_claims['num_procedures'] = all_claims[diag_cols + proc_cols].notna().sum(axis=1)

# Hospital stay days
if 'ClaimStartDt' in all_claims.columns and 'ClaimEndDt' in all_claims.columns:
    all_claims['ClaimEndDt'] = pd.to_datetime(all_claims['ClaimEndDt'], errors='coerce')
    all_claims['hospital_stay_days'] = (all_claims['ClaimEndDt'] - all_claims['ClaimStartDt']).dt.days
    all_claims['hospital_stay_days'] = all_claims['hospital_stay_days'].fillna(0).clip(lower=0)
else:
    all_claims['hospital_stay_days'] = 0

# Count previous claims per patient
all_claims = all_claims.sort_values(['BeneID', 'ClaimStartDt']) if 'ClaimStartDt' in all_claims.columns else all_claims
all_claims['num_previous_claims'] = all_claims.groupby('BeneID').cumcount()

# Provider claim count
all_claims['provider_claim_count'] = all_claims.groupby('Provider')['ClaimID'].transform('count')

# Create additional simple features
all_claims['diagnosis_complexity'] = all_claims['num_procedures'] / (all_claims['num_procedures'].max() + 1)
all_claims['treatment_cost_ratio'] = all_claims['claim_amount'] / (all_claims['claim_amount'].mean() + 1)

# Processing time (admission to claim)
if 'AdmissionDt' in all_claims.columns and 'ClaimStartDt' in all_claims.columns:
    all_claims['AdmissionDt'] = pd.to_datetime(all_claims['AdmissionDt'], errors='coerce')
    all_claims['claim_processing_time'] = (all_claims['ClaimStartDt'] - all_claims['AdmissionDt']).dt.days
    all_claims['claim_processing_time'] = all_claims['claim_processing_time'].fillna(15).clip(lower=0, upper=180)
else:
    all_claims['claim_processing_time'] = 15

# Geographic risk score (using state if available)
if 'State' in all_claims.columns:
    state_fraud_rate = all_claims.groupby('State')['PotentialFraud'].apply(
        lambda x: (x == 'Yes').mean() if 'PotentialFraud' in all_claims.columns else 0.1
    )
    all_claims['geographic_risk_score'] = all_claims['State'].map(state_fraud_rate).fillna(0.1)
else:
    all_claims['geographic_risk_score'] = 0.1

print("✓ Feature engineering completed!")
print(f"\nCreated features:")
feature_cols = ['claim_amount', 'patient_age', 'num_procedures', 'hospital_stay_days',
                'num_previous_claims', 'provider_claim_count', 'diagnosis_complexity',
                'treatment_cost_ratio', 'claim_processing_time', 'geographic_risk_score']
for col in feature_cols:
    if col in all_claims.columns:
        print(f"  ✓ {col}")

In [ ]:
# Combine inpatient and outpatient claims
# Add claim type indicator
inpatient_df['ClaimType'] = 'Inpatient'
outpatient_df['ClaimType'] = 'Outpatient'

# Combine claims
all_claims = pd.concat([inpatient_df, outpatient_df], ignore_index=True)

print(f"Total claims: {len(all_claims)}")
print(f"\nClaim Types Distribution:")
print(all_claims['ClaimType'].value_counts())

# Check for Provider column and potential fraud labels
if 'PotentialFraud' in all_claims.columns:
    print(f"\nFraud Distribution:")
    print(all_claims['PotentialFraud'].value_counts())
elif 'Provider' in all_claims.columns and 'PotentialFraud' in providers_df.columns:
    # Merge with provider data to get fraud labels
    all_claims = all_claims.merge(providers_df[['Provider', 'PotentialFraud']], 
                                   on='Provider', how='left')
    print(f"\nFraud Distribution (from providers):")
    print(all_claims['PotentialFraud'].value_counts())

## 2. Load Data from CSV Files

Load the medical insurance fraud dataset from the `data` folder which contains:
- **Inpatient Claims**: Hospital admission claims
- **Outpatient Claims**: Outpatient visit claims
- **Patients Data**: Patient demographics and information
- **Providers Data**: Healthcare provider information

In [ ]:
# Load datasets
print("Loading datasets from data folder...")

# Load claims data
inpatient_df = pd.read_csv('../data/inpatient_claims_data.csv')
outpatient_df = pd.read_csv('../data/outpatient_claims_data.csv')

# Load supporting data
patients_df = pd.read_csv('../data/patients_data.csv')
providers_df = pd.read_csv('../data/providers_data.csv')

print(f"✓ Loaded {len(inpatient_df)} inpatient claims")
print(f"✓ Loaded {len(outpatient_df)} outpatient claims")
print(f"✓ Loaded {len(patients_df)} patient records")
print(f"✓ Loaded {len(providers_df)} provider records")

# Display dataset info
print("\nInpatient Claims Columns:", inpatient_df.columns.tolist())
print("Outpatient Claims Columns:", outpatient_df.columns.tolist())

## 3. Data Preprocessing and Feature Engineering

We'll combine and process the data to create features suitable for the baseline model.

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Display first few rows
print("Sample Data:")
display(X.head(10))

# Statistical summary
print("\nStatistical Summary:")
display(X.describe())

# Check for missing values
print("\nMissing Values:")
print(X.isnull().sum())

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
y.value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class (0=Legitimate, 1=Fraudulent)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Legitimate', 'Fraudulent'], rotation=0)

# Pie chart
colors = ['#2ecc71', '#e74c3c']
axes[1].pie(y.value_counts(), labels=['Legitimate', 'Fraudulent'], 
            autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Class Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of key features by fraud status
features_to_plot = ['claim_amount', 'patient_age', 'num_procedures', 'hospital_stay_days']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(features_to_plot):
    # Legitimate claims
    axes[idx].hist(X[y==0][feature], bins=50, alpha=0.6, label='Legitimate', color='#2ecc71')
    # Fraudulent claims
    axes[idx].hist(X[y==1][feature], bins=50, alpha=0.6, label='Fraudulent', color='#e74c3c')
    
    axes[idx].set_xlabel(feature.replace('_', ' ').title(), fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'{feature.replace("_", " ").title()} Distribution', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 5. Train-Test Split

Split the data into training and testing sets (80/20 split).

In [ ]:
def create_basic_features(df):
    """Create basic engineered features for baseline model"""
    df = df.copy()
    
    # Simple ratio features (traditional tabular representation)
    df['cost_per_procedure'] = df['claim_amount'] / (df['num_procedures'] + 1)
    df['cost_per_day'] = df['claim_amount'] / (df['hospital_stay_days'] + 1)
    df['procedures_per_day'] = df['num_procedures'] / (df['hospital_stay_days'] + 1)
    
    return df

# Apply basic feature engineering
X_engineered = create_basic_features(X)

print(f"Original features: {X.shape[1]}")
print(f"Total features (with basic engineering): {X_engineered.shape[1]}")
print(f"New features added: {X_engineered.shape[1] - X.shape[1]}")
print("\nFeature approach: Traditional tabular representation")
print("✓ Basic feature engineering complete!")

## 6. Feature Scaling

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_engineered, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining set class distribution:")
print(f"  Legitimate: {(y_train==0).sum()} ({(y_train==0).sum()/len(y_train)*100:.2f}%)")
print(f"  Fraudulent: {(y_train==1).sum()} ({(y_train==1).sum()/len(y_train)*100:.2f}%)")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Data preprocessed and split successfully!")

## 7. Model Training - Baseline Stacking Ensemble

Train the baseline model using:
- **Base Learners**: Random Forest, Gradient Boosting, XGBoost, LightGBM
- **Meta-Learner**: Logistic Regression
- **Class Imbalance Handling**: Class weights (balanced)

In [ ]:
# Initialize base models
base_models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=4,
        max_features='sqrt',
        class_weight='balanced',
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),
    
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=7,
        min_samples_split=10,
        min_samples_leaf=4,
        subsample=0.8,
        random_state=RANDOM_STATE
    ),
    
    'XGBoost': XGBClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=7,
        min_child_weight=3,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=10,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        eval_metric='logloss'
    ),
    
    'LightGBM': LGBMClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=7,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        class_weight='balanced',
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )
}

print("✓ Base models initialized!")
print(f"\nTotal base models: {len(base_models)}")

In [ ]:
# Train all base models
print("=" * 70)
print("Training Base Models")
print("=" * 70)

trained_models = {}

for name, model in base_models.items():
    print(f"\n[Training] {name}...")
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    print(f"✓ {name} trained successfully!")

print("\n" + "=" * 70)
print("✓ All base models trained!")
print("=" * 70)

In [ ]:
# Create and train Stacking Classifier
print("\n" + "=" * 70)
print("[Training] Stacking Classifier (Meta-Learner Ensemble)...")
print("=" * 70)

# Define base estimators for stacking
estimators = [
    ('rf', trained_models['Random Forest']),
    ('gb', trained_models['Gradient Boosting']),
    ('xgb', trained_models['XGBoost']),
    ('lgb', trained_models['LightGBM'])
]

# Create stacking classifier with Logistic Regression as meta-learner
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=1000
    ),
    cv=5,  # 5-fold cross-validation
    stack_method='predict_proba',  # Use probability predictions
    n_jobs=-1
)

print('\nStacking architecture:')
print('  Base learners: Random Forest, Gradient Boosting, XGBoost, LightGBM')
print('  Meta-learner: Logistic Regression')
print('  CV strategy: 5-fold cross-validation')
print('\nTraining stacking ensemble...')

stacking_clf.fit(X_train_scaled, y_train)
trained_models['Stacking Classifier'] = stacking_clf

print("\n✓ Stacking Classifier trained successfully!")
print("=" * 70)
print(f"✓ All {len(trained_models)} models ready for evaluation!")
print("=" * 70)

## 8. Model Evaluation

Evaluate the trained model on the test set.

In [ ]:
# Evaluate all models
results = {}

for name, model in trained_models.items():
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate metrics
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba),
        'Avg Precision': average_precision_score(y_test, y_pred_proba)
    }

# Display results
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)
print("\n" + "=" * 90)
print("MODEL PERFORMANCE COMPARISON (Baseline - Traditional Features)")
print("=" * 90)
display(results_df)
print("=" * 90)

In [ ]:
# Detailed classification report for best model (Stacking Classifier)
best_model = trained_models['Stacking Classifier']
y_pred_best = best_model.predict(X_test_scaled)

print("\n" + "=" * 70)
print("DETAILED CLASSIFICATION REPORT - STACKING CLASSIFIER")
print("=" * 70)
print(classification_report(y_test, y_pred_best, 
                          target_names=['Legitimate', 'Fraudulent'],
                          digits=4))
print("=" * 70)

## 9. Visualizations

In [ ]:
# Model comparison bar chart
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(trained_models))
width = 0.15

for i, metric in enumerate(metrics_to_plot):
    values = [results[model][metric] for model in trained_models.keys()]
    offset = width * (i - len(metrics_to_plot) / 2)
    ax.bar(x + offset, values, width, label=metric)

ax.set_xlabel('Models', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Model Performance Comparison (Baseline - Traditional Features)', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(trained_models.keys(), rotation=15, ha='right')
ax.legend(loc='lower right')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, (name, model) in enumerate(trained_models.items()):
    y_pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Legitimate', 'Fraudulent'],
                yticklabels=['Legitimate', 'Fraudulent'],
                cbar_kws={'shrink': 0.8})
    
    axes[idx].set_title(f'{name}\nAccuracy: {results[name]["Accuracy"]:.4f}',
                       fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Predicted', fontsize=10)
    axes[idx].set_ylabel('Actual', fontsize=10)

# Hide the last subplot if odd number of models
if len(trained_models) < 6:
    axes[-1].axis('off')

plt.suptitle('Confusion Matrices - All Models (Baseline)', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves for all models
plt.figure(figsize=(12, 8))

for name, model in trained_models.items():
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    plt.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC = {auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random (AUC = 0.5000)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curves - All Models (Baseline)', fontsize=14, fontweight='bold', pad=20)
plt.legend(loc="lower right", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall Curves for all models
plt.figure(figsize=(12, 8))

for name, model in trained_models.items():
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    avg_precision = average_precision_score(y_test, y_pred_proba)
    
    plt.plot(recall, precision, linewidth=2, 
             label=f'{name} (AP = {avg_precision:.4f})')

plt.xlabel('Recall', fontsize=12, fontweight='bold')
plt.ylabel('Precision', fontsize=12, fontweight='bold')
plt.title('Precision-Recall Curves - All Models (Baseline)', fontsize=14, fontweight='bold', pad=20)
plt.legend(loc="lower left", fontsize=10)
plt.grid(alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance for tree-based models
tree_models = ['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM']

fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.ravel()

for idx, model_name in enumerate(tree_models):
    model = trained_models[model_name]
    
    # Get feature importances
    importances = model.feature_importances_
    feature_names = X_engineered.columns
    
    # Create dataframe and sort
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(15)
    
    # Plot
    axes[idx].barh(range(len(importance_df)), importance_df['importance'], color='steelblue')
    axes[idx].set_yticks(range(len(importance_df)))
    axes[idx].set_yticklabels(importance_df['feature'], fontsize=9)
    axes[idx].set_xlabel('Importance', fontsize=10, fontweight='bold')
    axes[idx].set_title(f'Top 15 Features - {model_name}', fontsize=11, fontweight='bold')
    axes[idx].invert_yaxis()
    axes[idx].grid(axis='x', alpha=0.3)

plt.suptitle('Feature Importance - Tree-Based Models (Baseline)', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 10. Summary and Conclusions

In [ ]:
# Find best performing model
best_model_name = results_df['F1-Score'].idxmax()
best_metrics = results_df.loc[best_model_name]

print("=" * 80)
print("FINAL SUMMARY - BASELINE MODEL (Traditional Tabular Features)")
print("=" * 80)
print("\n📊 Dataset Information:")
print(f"   Total samples: {len(X)}")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Features (original): {X.shape[1]}")
print(f"   Features (with basic engineering): {X_engineered.shape[1]}")
print(f"   Fraud ratio: {(y==1).sum()/len(y)*100:.2f}%")
print(f"   Imbalance handling: None (class_weight='balanced' in models)")

print("\n🏆 Best Performing Model: " + best_model_name)
print(f"   Accuracy:  {best_metrics['Accuracy']:.4f}")
print(f"   Precision: {best_metrics['Precision']:.4f}")
print(f"   Recall:    {best_metrics['Recall']:.4f}")
print(f"   F1-Score:  {best_metrics['F1-Score']:.4f}")
print(f"   ROC-AUC:   {best_metrics['ROC-AUC']:.4f}")

print("\n📈 Model Rankings (by F1-Score):")
rankings = results_df.sort_values('F1-Score', ascending=False)
for i, (model, metrics) in enumerate(rankings.iterrows(), 1):
    print(f"   {i}. {model}: {metrics['F1-Score']:.4f}")

print("\n✅ Key Characteristics:")
print("   • Traditional tabular feature representation")
print("   • Stacking ensemble with Logistic Regression meta-learner")
print("   • No synthetic data generation (SMOTE)")
print("   • Basic feature engineering (3 simple ratio features)")
print("   • Serves as baseline for comparison")

print("\n💡 Next Steps:")
print("   • Compare with SMOTE model for performance improvement")
print("   • Advanced feature engineering with relational patterns")
print("   • Hyperparameter tuning for optimization")
print("   • Explore different meta-learners for stacking")

print("\n" + "=" * 80)
print("✓ Baseline Model Analysis Complete!")
print("=" * 80)